In [1]:
import os
import re
import PyPDF2
import pdfplumber

import numpy as np
import pandas as pd

import torch
import evaluate
 
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset, Dataset


# Подготовка данных

In [2]:
data_vacancy_hh = pd.read_csv('hhparser_vacancy.csv', on_bad_lines='skip') #, delimiter='|')

In [3]:
data_vacancy_hh

,id,name,has_test,response_letter_required,salary_from,salary_to,salary_currency,salary_gross,published_at,created_at,parsed_at,url,employer_name,description,alternate_url,area_id,area_name
0,38228194,Project R’n’D / Технолог R’n’D,False,True,NaN,NaN,NaN,NaN,2020-07-26 23:40:42+00,2020-07-26 23:40:42+00,2020-07-27 03:03:03.154874+00,https://api.hh.ru/vacancies/38228194?host=hh.ru,Масл фектори,<p><strong>MF Kitchen</strong> — группа компан...,https://hh.ru/vacancy/38228194,1,Москва
1,36911636,"Продавец-кассир (Москва, Богданова, 16)",False,False,33500.0,40000.0,RUR,True,2020-07-27 00:28:12+00,2020-07-27 00:28:12+00,2020-07-27 03:03:04.258759+00,https://api.hh.ru/vacancies/36911636?host=hh.ru,Пятёрочка,<p><strong>Обязанности:</strong></p><ul><li>Об...,https://hh.ru/vacancy/36911636,1,Москва
2,36909000,"Продавец-кассир (Москва, Новогиреевская, 11/36)",False,False,33500.0,38000.0,RUR,True,2020-08-24 00:16:48+00,2020-08-24 00:16:48+00,2020-08-24 02:59:43.766137+00,https://api.hh.ru/vacancies/36909000?host=hh.ru,Пятёрочка,<p><strong>Обязанности:</strong></p><ul><li>Об...,https://hh.ru/vacancy/36909000,1,Москва
3,38985954,"Product owner бизнес–стрима ""Аналитика и Отчет...",False,False,NaN,NaN,NaN,NaN,2020-09-08 18:29:25+00,2020-09-08 18:29:25+00,2020-09-08 19:02:47.261747+00,https://api.hh.ru/vacancies/38985954?host=hh.ru,Банк ВТБ (ПАО),<p>Наша позиция предполагает участие в програм...,https://hh.ru/vacancy/38985954,1,Москва
4,40108800,Business Development Director (CTO),False,False,NaN,NaN,NaN,NaN,2020-11-25 08:56:45+00,2020-11-25 08:56:45+00,2020-11-28 06:59:37.872153+00,https://api.hh.ru/vacancies/40108800?host=hh.ru,Студия Олега Чулакова,<p>Студия Олега Чулакова — это дизайн-студия №...,https://hh.ru/vacancy/40108800,1,Москва
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96220,38834939,Оператор 1C,False,False,48000.0,NaN,RUR,True,2020-08-26 06:52:49+00,2020-08-26 06:52:49+00,2020-08-26 07:02:58.340756+00,https://api.hh.ru/vacancies/38834939?host=hh.ru,ТД ЛФБ,"<p> </p> <p>ТД «ЛФБ» - семейная компания, веде...",https://hh.ru/vacancy/38834939,1,Москва
96221,37345373,Стажер Управления финансирования торговли,False,False,40000.0,40000.0,RUR,True,2020-07-25 13:23:01+00,2020-07-25 13:23:01+00,2020-07-28 11:01:18.939989+00,https://api.hh.ru/vacancies/37345373?host=hh.ru,UniCredit Bank,<p>КРАТКАЯ ИНФОРМАЦИЯ:</p> <p>Отдел финансиров...,https://hh.ru/vacancy/37345373,1,Москва
96222,38602820,Преподаватель математики/информатики,False,False,NaN,NaN,NaN,NaN,2020-08-13 16:30:47+00,2020-08-13 16:30:47+00,2020-08-15 07:01:49.548725+00,https://api.hh.ru/vacancies/38602820?host=hh.ru,Центр дополнительного образования Новый Взгляд,<p><strong>Обязанности:</strong></p> <p> </p> ...,https://hh.ru/vacancy/38602820,1,Москва
96223,35870421,Программист С++ / Linux,False,False,NaN,NaN,NaN,NaN,2020-07-25 06:56:59+00,2020-07-25 06:56:59+00,2020-07-27 07:01:31.732133+00,https://api.hh.ru/vacancies/35870421?host=hh.ru,Военно-космическая академия имени А.Ф. Можайского,<p><strong>Требования:</strong></p> <ul> <li>З...,https://hh.ru/vacancy/35870421,2,Санкт-Петербург


In [4]:
data_vacancy_hh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96225 entries, 0 to 96224
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        96225 non-null  int64  
 1   name                      96225 non-null  object 
 2   has_test                  96225 non-null  bool   
 3   response_letter_required  96225 non-null  bool   
 4   salary_from               56084 non-null  float64
 5   salary_to                 39687 non-null  float64
 6   salary_currency           61763 non-null  object 
 7   salary_gross              61493 non-null  object 
 8   published_at              96225 non-null  object 
 9   created_at                96225 non-null  object 
 10  parsed_at                 96225 non-null  object 
 11  url                       96225 non-null  object 
 12  employer_name             96225 non-null  object 
 13  description               96223 non-null  object 
 14  altern

In [5]:
columns = data_vacancy_hh.columns
columns

Index(['id', 'name', 'has_test', 'response_letter_required', 'salary_from',
       'salary_to', 'salary_currency', 'salary_gross', 'published_at',
       'created_at', 'parsed_at', 'url', 'employer_name', 'description',
       'alternate_url', 'area_id', 'area_name'],
      dtype='object')

In [6]:
columns_hold = ['name', 'description']

In [27]:
data_vacancy = data_vacancy_hh[columns_hold]

In [28]:
data_vacancy

,name,description
0,Project R’n’D / Технолог R’n’D,<p><strong>MF Kitchen</strong> — группа компан...
1,"Продавец-кассир (Москва, Богданова, 16)",<p><strong>Обязанности:</strong></p><ul><li>Об...
2,"Продавец-кассир (Москва, Новогиреевская, 11/36)",<p><strong>Обязанности:</strong></p><ul><li>Об...
3,"Product owner бизнес–стрима ""Аналитика и Отчет...",<p>Наша позиция предполагает участие в програм...
4,Business Development Director (CTO),<p>Студия Олега Чулакова — это дизайн-студия №...
...,...,...
96220,Оператор 1C,"<p> </p> <p>ТД «ЛФБ» - семейная компания, веде..."
96221,Стажер Управления финансирования торговли,<p>КРАТКАЯ ИНФОРМАЦИЯ:</p> <p>Отдел финансиров...
96222,Преподаватель математики/информатики,<p><strong>Обязанности:</strong></p> <p> </p> ...
96223,Программист С++ / Linux,<p><strong>Требования:</strong></p> <ul> <li>З...


In [31]:
def replace_symb(str_):\
    return str(str_).replace('<p>', ' '). \
                replace('<strong>', ' '). \
                replace('</p>', ''). \
                replace('</strong>', ' '). \
                replace('<ul>', ''). \
                replace('</ul>', ''). \
                replace('</li>', ''). \
                replace('<li>', '')

In [32]:
data_vacancy.description = data_vacancy.description.apply(replace_symb)

C:\Users\basil\AppData\Local\Temp\ipykernel_10112\3785133548.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_vacancy.description = data_vacancy.description.apply(replace_symb)


In [33]:
data_vacancy

,name,description
0,Project R’n’D / Технолог R’n’D,"MF Kitchen — группа компаний, занимающаяся ..."
1,"Продавец-кассир (Москва, Богданова, 16)",Обязанности: Обслуживание покупателей на кас...
2,"Продавец-кассир (Москва, Новогиреевская, 11/36)",Обязанности: Обслуживание покупателей на кас...
3,"Product owner бизнес–стрима ""Аналитика и Отчет...",Наша позиция предполагает участие в программе...
4,Business Development Director (CTO),Студия Олега Чулакова — это дизайн-студия №1 ...
...,...,...
96220,Оператор 1C,"ТД «ЛФБ» - семейная компания, ведет свою и..."
96221,Стажер Управления финансирования торговли,КРАТКАЯ ИНФОРМАЦИЯ: Отдел финансирования тор...
96222,Преподаватель математики/информатики,Обязанности: подготовка и проведение гр...
96223,Программист С++ / Linux,Требования: Знание С/С++/C++11 Опыт работы...


In [34]:
data_vacancy = data_vacancy.dropna(how='any')
data_vacancy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96225 entries, 0 to 96224
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         96225 non-null  object
 1   description  96225 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


In [43]:
data_vacancy = data_vacancy.rename(columns={"name": "Summaries", "description": "Articles"})

# модель

In [36]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-absum'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [79]:
ds_summarize = Dataset.from_pandas(data_vacancy)

C:\anaconda3\lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [80]:
ds_summarize

Dataset({
    features: ['Summaries', 'Articles'],
    num_rows: 96225
})

In [81]:
ds_summarize = ds_summarize.train_test_split(train_size=0.8)

In [82]:
ds_summarize

DatasetDict({
    train: Dataset({
        features: ['Summaries', 'Articles'],
        num_rows: 76980
    })
    test: Dataset({
        features: ['Summaries', 'Articles'],
        num_rows: 19245
    })
})

In [89]:
MODEL = 't5-base'
BATCH_SIZE = 2
EPOCHS = 3
OUT_DIR = 'results_t5base'
MAX_INPUT_LENGTH = 512
MAX_LENGTH = 64 

In [84]:
# Function to convert text data into model inputs and targets
def preprocess_function(examples):
    inputs = [f"summarize: {article}" for article in examples['Articles']]
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        padding='max_length'
    )
 
    # Set up the tokenizer for targets
    targets = [summary for summary in examples['Summaries']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=MAX_LENGTH,
            truncation=True,
            padding='max_length'
        )
 
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
 
# Apply the function to the whole dataset
ds_tokenized = ds_summarize.map(preprocess_function, batched=True)

Map:   0%|          | 0/76980 [00:00<?, ? examples/s]

C:\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3660: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/19245 [00:00<?, ? examples/s]

In [85]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

244,309,248 total parameters.
244,309,248 training parameters.


In [86]:
rouge = evaluate.load("rouge")
 
def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids
 
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
 
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )
 
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
 
    return {k: round(v, 4) for k, v in result.items()}

In [87]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [91]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    #logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=10000,
    save_strategy='epoch',
    save_total_limit=2,
    learning_rate=0.0001
)
 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_tokenized["train"],
    eval_dataset=ds_tokenized["test"],
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)
 
history = trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Gen Len
10000,0.238500,0.224043,0.186000,0.070000,0.185200,9.715600
20000,0.232700,0.207933,0.199600,0.074600,0.198700,9.715700
30000,0.225200,0.197720,0.196400,0.072200,0.195800,9.715700
40000,0.183900,0.191968,0.212300,0.083300,0.211600,9.715700
50000,0.189300,0.186672,0.210800,0.082900,0.210100,9.715600
60000,0.175500,0.182822,0.212400,0.088800,0.211700,9.715700
70000,0.174600,0.179088,0.216500,0.089300,0.216000,9.715700
80000,0.162200,0.178113,0.221000,0.092400,0.220200,9.715700
90000,0.151000,0.176523,0.221100,0.094500,0.220300,9.715700
100000,0.153600,0.173735,0.227900,0.100300,0.227100,9.715700


In [96]:
model.save_pretrained('t5_vacancy_3epochs')
tokenizer.save_pretrained('t5_vacancy_tokenizer')

('t5_vacancy_tokenizer\\tokenizer_config.json',
 't5_vacancy_tokenizer\\special_tokens_map.json',
 't5_vacancy_tokenizer\\spiece.model',
 't5_vacancy_tokenizer\\added_tokens.json')

In [ ]:
#model = T5ForConditionalGeneration.from_pretrained(model_path)
#tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)

# Тест

In [92]:
def summarize_text(text, model, tokenizer, num_beams=5):
    # Preprocess the text
    inputs = tokenizer.encode(
        "summarize: " + text,
        return_tensors='pt',
        max_length=1024,
        truncation=True
    )
 
    # Generate the summary
    summary_ids = model.generate(
        inputs,
        max_length=64,
        num_beams=num_beams,
        # early_stopping=True,
    )
 
    # Decode and return the summary
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [95]:
model_ = model.to('cpu')
text = data_vacancy.iloc[30000].Articles #ds_summarize['test'][100]['Articles']
summary = summarize_text(text=text,
              model=model_,
              tokenizer=tokenizer) 
print('text: ', text)
print('summary: ', summary)

text:   Крупная компания ищет сотрудников на должности: Уборщица(к) м. Бабушкинская   Обязанности:     Влажная и сухая уборка помещений    Требования:     Трудолюбие и исполнительность    Работоспособность    Ответственность     Условия:   - Стабильная работа  - График работы: 5/2 с 8-00 до 12-00  - Заработная плата без задержек   
summary:  Уборщица/уборщик


In [97]:
data_vacancy.iloc[30000].Summaries

'Уборщица/уборщик'

In [100]:
text = 'Работа на проектах внедрения ИС на платформе SAP BO FC/IC (бюджетирование, консолидация финансовой отчетности; интеграция Excel - SAP FC -SAP IC - SAP BPC - BW); - руководство группой тех.поддержки; - FIM/DS jobs - разработка//поддержка (ETL, модификация FIM jobs в DS Designer - scripts, custom functions, SQL queries/sp, интеграция между различными системами); - отчеты Crystal Reports; - MS SQL 2016(поддержка модифицированных FIM jobs, разработка запросов, хранимых процедур, udf, запросы к БД FIM/FC/IC/BOE audit); - модификация/поддержка макросов в Excel VBA / VBS (подготовка исходных файлов Excel для загрузки в SAP IC/FC); - внутренний проект - "База данных пользователей КИС" (MS SQL / C#, MS Visual Studio 2013, MS SQL Server 2008, интеграция с системами SAP - формирование файлов загрузки для SAP NW, BPC, IC).'
text += 'C/C++ MS SQL Server Transact-SQL VBA Python SQL Data Science Big Data Машинное обучение Глубокое обучение Numpy SciPy PyTorch Keras Scikit-learn Pandas SkLearn'
summary = summarize_text(text=text,
              model=model_,
              tokenizer=tokenizer) 
print('text: ', text)
print('summary: ', summary)

text:  Работа на проектах внедрения ИС на платформе SAP BO FC/IC (бюджетирование, консолидация финансовой отчетности; интеграция Excel - SAP FC -SAP IC - SAP BPC - BW); - руководство группой тех.поддержки; - FIM/DS jobs - разработка//поддержка (ETL, модификация FIM jobs в DS Designer - scripts, custom functions, SQL queries/sp, интеграция между различными системами); - отчеты Crystal Reports; - MS SQL 2016(поддержка модифицированных FIM jobs, разработка запросов, хранимых процедур, udf, запросы к БД FIM/FC/IC/BOE audit); - модификация/поддержка макросов в Excel VBA / VBS (подготовка исходных файлов Excel для загрузки в SAP IC/FC); - внутренний проект - "База данных пользователей КИС" (MS SQL / C#, MS Visual Studio 2013, MS SQL Server 2008, интеграция с системами SAP - формирование файлов загрузки для SAP NW, BPC, IC).C/C++ MS SQL Server Transact-SQL VBA Python SQL Data Science Big Data Машинное обучение Глубокое обучение Numpy SciPy PyTorch Keras Scikit-learn Pandas SkLearn
summary:  В

In [101]:
text = 'Входной контроль сырья и комплектующих, контроль тех.процесса изготовления продукции, концевые операции.'
text += 'Приемка, продажа товара. Работа на контрольно-кассовой технике. Соблюдение требований стандартов предприятия'
text += 'Пользователь ПК Водительское удостоверение категории B Организаторские навыки Организация мероприятий Холодные продажи Делопроизводство Поиск и привлечение клиентов Заключение договоров Телефонные переговоры Прямые продажи Windows 7'
summary = summarize_text(text=text,
              model=model_,
              tokenizer=tokenizer) 
print('text: ', text)
print('summary: ', summary)

text:  Входной контроль сырья и комплектующих, контроль тех.процесса изготовления продукции, концевые операции.Приемка, продажа товара. Работа на контрольно-кассовой технике. Соблюдение требований стандартов предприятияПользователь ПК Водительское удостоверение категории B Организаторские навыки Организация мероприятий Холодные продажи Делопроизводство Поиск и привлечение клиентов Заключение договоров Телефонные переговоры Прямые продажи Windows 7
summary:  Менеджер по продажам и работе с клиентами


In [102]:
text = 'Организация и контроль рабочего процесса Эксплуатация зданий и сооружений Ремонтные работы Техническое обслуживание Энергетика Первичная бухгалтерская документация Работа с электронным документооборотом Договорная работа Оформление ведомости объёмов строительных, электромонтажных работ Работа с технической документацией Техническая эксплуатация Ведение переговоров Противопожарная безопасность Монтаж оборудования Административно-хозяйственная деятельность'

summary = summarize_text(text=text,
              model=model_,
              tokenizer=tokenizer) 
print('text: ', text)
print('summary: ', summary)


text:  Организация и контроль рабочего процесса Эксплуатация зданий и сооружений Ремонтные работы Техническое обслуживание Энергетика Первичная бухгалтерская документация Работа с электронным документооборотом Договорная работа Оформление ведомости объёмов строительных, электромонтажных работ Работа с технической документацией Техническая эксплуатация Ведение переговоров Противопожарная безопасность Монтаж оборудования Административно-хозяйственная деятельность
summary:  Руководитель отдела эксплуатации зданий и сооружений
